In [1]:
import streamlit as st

from langchain_openai import AzureChatOpenAI, AzureOpenAI
from langchain_openai import AzureOpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.document_loaders import PyPDFLoader
from langchain_pinecone import PineconeVectorStore
from pinecone import Pinecone

import os

from dotenv import load_dotenv
load_dotenv()

f:\DATA\CODING\LANGCHAIN\venv\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


True

In [2]:
embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-3-large",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    api_key=os.getenv("OPENAI_API_KEY")
)
endpoint=os.getenv("AZURE_CHAT_OPENAI_ENDPOINT")
key=os.getenv("OPENAI_API_KEY")
llm = AzureChatOpenAI(
    deployment_name="gpt-4o",
    model_name="gpt-4o",
    azure_endpoint=endpoint,
    api_version="2023-03-15-preview",
    openai_api_key=key,
)

In [3]:
documents=[]
loader=PyPDFLoader("social science 32-2-1 Solution.pdf")
docs=loader.load()
documents.extend(docs)

f:\DATA\CODING\LANGCHAIN\venv\lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from this module in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [4]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=5000, chunk_overlap=500)
splits = text_splitter.split_documents(documents) 

index_name = os.getenv("PINECONE_INDEX") # put in the name of your pinecone index here

pc = Pinecone(
api_key = os.getenv("PINECONE_API_KEY"),
)

pc.list_indexes().names() # to check if my index exsist
index = pc.Index(index_name)
index.describe_index_stats()

vectorstore = PineconeVectorStore(pinecone_api_key = os.getenv("PINECONE_API_KEY"), index_name= index_name, embedding=embeddings, index= index)
vectorstore.add_texts(texts=[t.page_content for t in splits])
retriever = vectorstore.as_retriever()

In [5]:
true_answer_prompt = """
You are a teacher. You will answer the following question. The points that should be present in your answer is present in the context provided. You must strictly follow it. You should not give the answer outside the context. However, You have to elaborate the points with your own understanding. You have to answer the question pointwise.

Question: {question} 
Context: {context} 
Answer:
"""

true_answer_generator = ChatPromptTemplate.from_messages([
("human",true_answer_prompt),
])

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | true_answer_generator
    | llm
    | StrOutputParser()
)

In [6]:
question = "Question No: 32 (b). Democracy accommodates social diversities. Analyse. It is a 5 mark question."
true_answer = rag_chain.invoke(question)

In [7]:
print(true_answer)

**Question No: 32 (b). Democracy accommodates social diversities. Analyse.**

1. **Peaceful and Harmonious Life:**
   - **Explanation:** Democracy helps in the accommodation of social diversity, allowing citizens to lead peaceful and harmonious lives. This is achieved by ensuring that different social groups can coexist and work together within the same political framework.
   - **Example:** Belgium is a prime example where democracy has successfully negotiated differences among ethnic populations, including Dutch-speaking Flemish and French-speaking Walloons, to maintain national unity and peace.

2. **Procedures to Accommodate Competition:**
   - **Explanation:** Democracies typically develop procedures to accommodate competition among various social groups. These mechanisms are essential to ensure that all groups have a platform to express their interests and compete fairly.
   - **Example:** In India, the reservation system in education and employment for Scheduled Castes, Schedule

In [8]:
with open("student_answer.txt", "r") as file:
    # Read the entire content into a variable
    student_answer = file.read()


In [23]:
grading_criteria = """

1) Relevance to the context of the question
2) Examples and Illustrations

Any repetition or duplicate points shall not fetch any mark.

"""

evaluaton_analysis = """

1) Student's Answer Breakdown along with feedback
2) Final Score along with breakup

"""

In [24]:
answer_evaluation_prompt = f"""
You are a teacher who will evaluate and grade an exam. You are given a question and the student's answer. You have to evaluate the answer with respect to the true answer and give marks as per the relevance to the true answer. You will give marks as following the Grading Criteria. You will give a short analysis of the answer which contains: 

Question: {question} 
Student's answer: {student_answer} 
True Answer: {true_answer}
Grading Criteria: {grading_criteria}
Evaluation Analysis: {evaluaton_analysis}
Answer:
"""

# true_answer_generator = ChatPromptTemplate.from_messages([
# ("human",true_answer_prompt),
# ])


# ## chain 
# chain=true_answer_generator|llm

response=llm.invoke(answer_evaluation_prompt)
print(response.content)



# def format_docs(docs):
#     return "\n\n".join(doc.page_content for doc in docs)



# true_answer_chain = (
#     { | format_docs, "question": RunnablePassthrough()}
#     | true_answer_generator
#     | llm
#     | StrOutputParser()
# )



**Evaluation Analysis:**

### Student's Answer Breakdown and Feedback:

1. **Technological advancements and globalisation (Paragraph 1 & 3):**
   - **Relevance to the context:** The discussion on technological advancements and their role in globalisation is not relevant to the question about democracy accommodating social diversities. 
   - **Examples and Illustrations:** The examples given are about transportation and communication improvements, which do not pertain to the mechanisms democracy employs to manage social diversities.

2. **Institutional mechanisms like dialogue, negotiation, and legislation (Paragraph 2 & 4):**
   - **Relevance to the context:** These points are relevant to the question as they discuss how democratic systems have institutional mechanisms to address social differences.
   - **Examples and Illustrations:** The example of the National Commission for Scheduled Castes and Scheduled Tribes in India is appropriate and illustrates how democracy can help marginal

In [ ]:
"""
You are a teacher grading a quiz.
You are given a question, the student’s answer, and the points, that should be in the student answer, and are asked to score the student answer as either CORRECT or INCORRECT

Example Format:
QUESTION: question here
STUDENT ANSWER: student’s answer here
POINTS IN THE ANSWER: true answer here
GRADE: CORRECT or INCORRECT here”
"""

'\nYou are a teacher grading a quiz.\nYou are given a question, the student’s answer, and the points, that should be in the student answer, and are asked to score the student answer as either CORRECT or INCORRECT\n'

In [52]:
"""
Human

You are question-answering assistant tasked with answering questions based on the provided context.

Here is the question: \

{question}

Use the following pieces of context to answer the question at the end. Use three sentences maximum. \

{context}

### Assistant

Answer: Think step by step.
"""

'\nHuman\n\nYou are question-answering assistant tasked with answering questions based on the provided context.\n\nHere is the question: \n{question}\n\nUse the following pieces of context to answer the question at the end. Use three sentences maximum. \n{context}\n\n### Assistant\n\nAnswer: Think step by step.\n'

In [53]:
"""
You are a student who read the context and is now answering the question for an exam. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Answer in 500 words if question is of 5 marks. Answer in 200 words if question is of 2 marks. Answer only using the context provided. Answer point wise.

"""

"\nYou are a student who read the context and is now answering the question for an exam. Use the following pieces of retrieved context to answer the question. If you don't know the answer, just say that you don't know. Answer in 500 words if question is of 5 marks. Answer in 200 words if question is of 2 marks. Answer only using the context provided. Answer point wise.\n\n"